In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchvision import models

from skimage.util import random_noise

import sys

In [2]:
batch_size = 128

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(444)
if device == 'cuda':
    torch.cuda.manual_seed_all(444)
print(device + ' is avaulable')

cuda is avaulable


In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
num_classes = 10

Files already downloaded and verified


In [5]:
## one-hot encoding

In [6]:
label_encoding = torch.FloatTensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                                    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                                    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                                    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],])

In [7]:
label_encoding = label_encoding.cuda()

In [8]:
path = 'TrainedModel/Resnet18(OneHotEncoding).pt'
resnet18 = torch.load(path)
resnet18.eval()
resnet18 = resnet18.cuda()

In [9]:
## negative

In [10]:
with torch.no_grad():
    negative_correct = 0
    negative_total = 0
    negative_acc = 0
    for i, (test_data, test_label) in enumerate(testloader):

        test_data = test_data.cuda()
        test_label = test_label.cuda()
        
        negative_test_data = 0 - test_data

        test_output = resnet18(negative_test_data)

        negative_total = negative_total + len(test_label)
        negative_correct = negative_correct + (test_output.max(1)[1] == test_label).sum().item()
        
    negative_acc = negative_correct / negative_total * 100
    print("Negative data test acc : {:3.3f}%".format(negative_acc))

Negative data test acc : 41.120%


In [11]:
## Gaussian noise

In [12]:
gaussian_noise_mean = 0
gaussian_noise_var = [i ** 2 * 0.01 for i in range(10)]
gaussian_noise_correct = [0 for i in range(10)]
gaussian_noise_total = [0 for i in range(10)]
gaussian_noise_acc = [0 for i in range(10)]

In [13]:
with torch.no_grad():
    for i in range(10):
        for _, (test_data, test_label) in enumerate(testloader):
            
            gaussian_noise_test_data = torch.tensor(random_noise(test_data, mode = 'gaussian', mean = gaussian_noise_mean, var = gaussian_noise_var[i], clip = True))
            
            gaussian_noise_test_data = gaussian_noise_test_data.to(device, dtype=torch.float)
            test_label = test_label.cuda()

            test_output = resnet18(gaussian_noise_test_data)

            gaussian_noise_total[i] = gaussian_noise_total[i] + len(test_label)
            gaussian_noise_correct[i] = gaussian_noise_correct[i] + (test_output.max(1)[1] == test_label).sum().item()
            
        gaussian_noise_acc[i] = gaussian_noise_correct[i] / gaussian_noise_total[i] * 100
        print("Gaussian noise data (var : {:.3f}) test acc : {:3.3f}%".format(gaussian_noise_var[i], gaussian_noise_correct[i] / gaussian_noise_total[i] * 100))

Gaussian noise data (var : 0.000) test acc : 80.380%
Gaussian noise data (var : 0.010) test acc : 75.670%
Gaussian noise data (var : 0.040) test acc : 56.110%
Gaussian noise data (var : 0.090) test acc : 36.680%
Gaussian noise data (var : 0.160) test acc : 23.890%
Gaussian noise data (var : 0.250) test acc : 17.410%
Gaussian noise data (var : 0.360) test acc : 13.930%
Gaussian noise data (var : 0.490) test acc : 12.540%
Gaussian noise data (var : 0.640) test acc : 11.960%
Gaussian noise data (var : 0.810) test acc : 11.350%


In [14]:
## Gaussian blur

In [15]:
gaussian_blur_sigma = [2 - i ** 2 * 0.02 for i in range(10)]
gaussian_blur_correct = [0 for i in range(10)]
gaussian_blur_total = [0 for i in range(10)]
gaussian_blur_acc = [0 for i in range(10)]

In [16]:
with torch.no_grad():
    for i in range(10):
        gaussian_blur_transform = transforms.Compose([transforms.GaussianBlur(kernel_size = 5, sigma = gaussian_blur_sigma[i])])
        for _, (test_data, test_label) in enumerate(testloader):
            
            gaussian_blur_test_data = gaussian_blur_transform(test_data)
            
            gaussian_blur_test_data = gaussian_blur_test_data.to(device, dtype=torch.float)
            test_label = test_label.cuda()

            test_output = resnet18(gaussian_blur_test_data)

            gaussian_blur_total[i] = gaussian_blur_total[i] + len(test_label)
            gaussian_blur_correct[i] = gaussian_blur_correct[i] + (test_output.max(1)[1] == test_label).sum().item()
        
        gaussian_blur_acc[i] = gaussian_blur_correct[i] / gaussian_blur_total[i] * 100
        print("Gaussian blur data (sigma : {:.3f}) test acc : {:3.3f}%".format(gaussian_blur_sigma[i], gaussian_blur_acc[i]))

Gaussian blur data (sigma : 2.000) test acc : 37.030%
Gaussian blur data (sigma : 1.980) test acc : 37.080%
Gaussian blur data (sigma : 1.920) test acc : 37.170%
Gaussian blur data (sigma : 1.820) test acc : 37.450%
Gaussian blur data (sigma : 1.680) test acc : 37.870%
Gaussian blur data (sigma : 1.500) test acc : 39.270%
Gaussian blur data (sigma : 1.280) test acc : 42.670%
Gaussian blur data (sigma : 1.020) test acc : 50.950%
Gaussian blur data (sigma : 0.720) test acc : 66.790%
Gaussian blur data (sigma : 0.380) test acc : 80.060%


In [17]:
## Salt and Pepper

In [18]:
salt_and_pepper_val = 0.5
salt_and_pepper_amount = [i ** 2 * 0.01 for i in range(10)]
salt_and_pepper_correct = [0 for i in range(10)]
salt_and_pepper_total = [0 for i in range(10)]
salt_and_pepper_acc = [0 for i in range(10)]

In [19]:
with torch.no_grad():
    for i in range(10):
        for _, (test_data, test_label) in enumerate(testloader):
            
            salt_and_pepper_test_data = torch.tensor(random_noise(test_data, mode = 's&p', amount = salt_and_pepper_amount[i], salt_vs_pepper = salt_and_pepper_val, clip = True))
            
            salt_and_pepper_test_data = salt_and_pepper_test_data.to(device, dtype=torch.float)
            test_label = test_label.cuda()

            test_output = resnet18(salt_and_pepper_test_data)

            salt_and_pepper_total[i] = salt_and_pepper_total[i] + len(test_label)
            salt_and_pepper_correct[i] = salt_and_pepper_correct[i] + (test_output.max(1)[1] == test_label).sum().item()

        salt_and_pepper_acc[i] = salt_and_pepper_correct[i] / salt_and_pepper_total[i] * 100
        print("Salt and Pepper data (amount : {:.3f}) test acc : {:3.3f}%".format(salt_and_pepper_amount[i], salt_and_pepper_acc[i]))

Salt and Pepper data (amount : 0.000) test acc : 80.380%
Salt and Pepper data (amount : 0.010) test acc : 72.960%
Salt and Pepper data (amount : 0.040) test acc : 51.430%
Salt and Pepper data (amount : 0.090) test acc : 28.450%
Salt and Pepper data (amount : 0.160) test acc : 16.590%
Salt and Pepper data (amount : 0.250) test acc : 12.780%
Salt and Pepper data (amount : 0.360) test acc : 11.480%
Salt and Pepper data (amount : 0.490) test acc : 10.670%
Salt and Pepper data (amount : 0.640) test acc : 10.730%
Salt and Pepper data (amount : 0.810) test acc : 10.070%


In [20]:
## FGSM (Fast Gradient Signed Method)

In [21]:
fgsm_epsilons = [i ** 2 * 0.001 for i in range(10)]
fgsm_correct = [0 for i in range(10)]
fgsm_total = [0 for i in range(10)]
fgsm_acc = [0 for i in range(10)]

In [22]:
for i in range(10):
    for _, (test_data, test_label) in enumerate(testloader):

        test_data = test_data.cuda()
        test_label = test_label.cuda()
        test_one_hot_label = label_encoding[test_label]
        
        test_data.requires_grad = True

        fgsm_test_data = resnet18(test_data)
        fgsm_pred = fgsm_test_data.max(1, keepdim = True)[1]

        fgsm_loss = F.mse_loss(fgsm_test_data, test_one_hot_label)
        resnet18.zero_grad()
        fgsm_loss.backward()

        test_data_grad = test_data.grad.data

        sign_test_data_grad = test_data_grad.sign()
        FGSM_images = test_data + fgsm_epsilons[i] * sign_test_data_grad
        FGSM_images = torch.clamp(FGSM_images, -1, 1)

        test_output = resnet18(FGSM_images)

        fgsm_total[i] = fgsm_total[i] + len(test_label)
        fgsm_correct[i] = fgsm_correct[i] + (test_output.max(1)[1] == test_label).sum().item()

    fgsm_acc[i] = fgsm_correct[i] / fgsm_total[i] * 100
    print("FGSM data (epsilons : {:.3f}) test acc : {:3.3f}%".format(fgsm_epsilons[i], fgsm_acc[i]))

FGSM data (epsilons : 0.000) test acc : 80.380%
FGSM data (epsilons : 0.001) test acc : 75.710%
FGSM data (epsilons : 0.004) test acc : 63.370%
FGSM data (epsilons : 0.009) test acc : 52.270%
FGSM data (epsilons : 0.016) test acc : 45.020%
FGSM data (epsilons : 0.025) test acc : 39.970%
FGSM data (epsilons : 0.036) test acc : 36.170%
FGSM data (epsilons : 0.049) test acc : 33.290%
FGSM data (epsilons : 0.064) test acc : 31.290%
FGSM data (epsilons : 0.081) test acc : 29.750%


In [23]:
## 결과 저장

In [24]:
negative_f = open("result/Negative(OneHotEncoding).txt", "w")
gaussian_noise_f = open("result/GaussianNoise(OneHotEncoding).txt", "w")
gaussian_blur_f = open("result/GaussianBlur(OneHotEncoding).txt", "w")
salt_and_pepper_f = open("result/SaltAndPepper(OneHotEncoding).txt", "w")
fgsm_f = open("result/FGSM(OneHotEncoding).txt", "w")

print(negative_acc, file = negative_f)

print(gaussian_noise_var, file = gaussian_noise_f)
print(gaussian_noise_acc, file = gaussian_noise_f)

print(gaussian_blur_sigma, file = gaussian_blur_f)
print(gaussian_blur_acc, file = gaussian_blur_f)

print(salt_and_pepper_amount, file = salt_and_pepper_f)
print(salt_and_pepper_acc, file = salt_and_pepper_f)

print(fgsm_epsilons, file = fgsm_f)
print(fgsm_acc, file = fgsm_f)

fgsm_f.close()
salt_and_pepper_f.close()
gaussian_blur_f.close()
gaussian_noise_f.close()
negative_f.close()